#### Digital Signal Processing Courseware: An Introduction (copyright © 2024)
## Authors: J. Christopher Edgar and Gregory A. Miller

Conversion from Mathematica to Jupyter Notebook by Song Liu.

The authors of this book are indebted to Prof. Bruce Carpenter (University of Illinois Urbana-Champaign). Bruce inspired the creation of this courseware, he consulted with the authors as this courseware was being developed, and he provided the original version of the code and text for several sections of this courseware (e.g. the section on complex numbers and the section on normal distributions). 

# <font color=red>DSP.04 Convolution and Filtering - Spatial Domain</font>

# <font color=red>Give it a Try!</font>
# <font color=red>Part 2</font>

### Setup

In [ ]:
# general imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image as img
from matplotlib import cm
from mpl_toolkits import mplot3d
from scipy.fft import fft, fftfreq
import matplotlib.patches as patches
import math
import cmath
import pandas as pd
from sympy import Symbol, sin, series
from sympy import roots, solve_poly_system
import scipy.special

import warnings
warnings.filterwarnings('ignore')

# Figure size 
plt.rc("figure", figsize=(8, 6))

#function to create time course figure
#one waveform
def make_plot_1(x1,y1,type="b",linewidth = 1): 
    plt.plot(x1, y1,type)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
#two overlaid waveforms with red and blue   
def make_plot_2(x1,y1,type1,x2,y2,type2): 
    plt.plot(x1, y1, type1)
    plt.plot(x2, y2, type2)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
#three overlaid waveforms with red, blue and green   
def make_plot_3(x1,y1,type1,x2,y2,type2,x3,y3,type3): 
    plt.plot(x1, y1, type1)
    plt.plot(x2, y2, type2)
    plt.plot(x3, y3, type3)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
def make_plot_3d(ax,x,y,z):    
    ax.contour3D(x, y, z, 50, cmap=cm.coolwarm)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
def make_plot_freq_1(x1,sample_rate, duration=1): 
    N = sample_rate * duration
    Nhalf = math.ceil(N/2)
    yf = fft(x1)
    xf = fftfreq(N, 1 / sample_rate)
    yf = yf[0:Nhalf]
    xf = xf[0:Nhalf]
    plt.plot(xf, np.abs(yf))
    
#two spectrums
def make_plot_freq_2(x1,x2,sample_rate, duration=1): 
    N = sample_rate * duration
    Nhalf = math.ceil(N/2)
    yf1 = fft(x1)
    yf2 = fft(x2)
    xf = fftfreq(N, 1 / sample_rate)

    yf1 = yf1[0:Nhalf]
    yf2 = yf2[0:Nhalf]
    xf = xf[0:Nhalf]

    plt.plot(xf, np.abs(yf1))
    plt.plot(xf, np.abs(yf2), color = 'r')
    
def make_imshow(x):
    plt.imshow(x,cmap='Greys_r')
    plt.tick_params(labelbottom = False, bottom = False)
    plt.tick_params(labelleft = False, left = False)
    
def make_imshow_color(x):
    plt.imshow(x)
    plt.tick_params(labelbottom = False, bottom = False)
    plt.tick_params(labelleft = False, left = False)
    
def round_complex(x):
    return complex(np.round(x.real,4),np.round(x.imag,4))

## <font color=red>DSP.04.G2) Characterizing Noise</font>

### <font color=red>DSP.04.G2.a) Calculating percentages</font>

In several previous sections, noise was added to timeseries and spatial patterns.

Check out this timeseries of 30,000 random points (don't worry if it takes a few seconds to generate the
plot - it is a lot of points).

In [ ]:
x = np.arange(0, 30000, 1)
noise = np.random.normal(0,1.5,30000)

plt.scatter(x, noise, c ="red", s=1)
plt.text(32000,0,'time',fontsize=15)
plt.text(0,5,'amplitude',fontsize=15)
 
# To show the plot
plt.show()

The distribution of the noise values above is obtained using the Python 'random.normal'
command.

The noise dataset above was created by setting the mean value to 0 and the standard deviation value
to 1.5 (values highlighted in red).

Here is the bell curve associated with those distribution values.

In [ ]:
# A custom function to calculate probability distribution function 
def pdf(x,m,std):      
    y_out = 1/(std * np.sqrt(2 * np.pi)) * np.exp( - (x - m)**2 / (2 * std**2)) 
    return y_out 

# To generate an array of x-values 
x = np.arange(-5, 5, 0.1) 

# To generate an array of 
# y-values using corresponding x-values 
y = pdf(x,0,1.5) 

# Plotting the bell-shaped curve 
plt.plot(x, y) 
plt.show()

Enter the correct values in the equation below to calculate the percentage of noise datapoints falling
+/- 1 SDs from the mean. [#Miller The next 3 exercises already had the correct values entered. We could keep them and add some brief narration about why they're correct. For the first one: "As we discussed earlier, the portion of the Gaussian (a.k.a. normal a.k.a. bell) curve between 1 standard deviation below the mean and 1 standard deviation above the mean is about 68%. Run the next code cell to verify that, with entries of 1.5 for the standard deviation." Or we could ask: "Do you recall how much of the Gaussian (a.k.a. normal a.k.a. bell) curve is between 1 standard deviation below the mean and 1 standard deviation above the mean? Run the next code cell to confirm." **Or we could remove a couple of values, so the user does have to fill them in. Ditto the other two examples.]**

In [ ]:
from scipy.stats import norm

round(100 * (norm.cdf(1.5, loc=0, scale=1.5) - norm.cdf(-1.5, loc=0, scale=1.5)),2)

Enter the correct values in the equation below to calculate the percentage of noise datapoints falling
+/- 2 SDs from the mean.

In [ ]:
from scipy.stats import norm

round(100 * (norm.cdf(1.5*2, loc=0, scale=1.5) - norm.cdf(-1.5*2, loc=0, scale=1.5)),2)

Enter the correct values in the equation below to calculate the percentage of noise datapoints falling
+/- 3 SDs from the mean.

In [ ]:
from scipy.stats import norm

round(100 * (norm.cdf(1.5*3, loc=0, scale=1.5) - norm.cdf(-1.5*3, loc=0, scale=1.5)),2)

### <font color=red>DSP.04.G2.b) Calculating percentages</font>

In several examples, noise was added to timeseries and spatial patterns.

Check out this timeseries of 30,000 random points (don't worry if it takes a few seconds to generate the
plot - it is a lot of points).

In [ ]:
x = np.arange(0, 30000, 1)
noise = np.random.normal(4,3,30000)

plt.scatter(x, noise, c ="red",s=1)
plt.text(32000,4,'time',fontsize=15)
plt.text(0,15,'amplitude',fontsize=15)
 
# To show the plot
plt.show()

The distribution of the noise values above is obtained using the Python 'random.normal'
command.

The noise dataset above was created by setting the mean value to 4 and the standard deviation value
to 3.

Here is the bell curve associated with those distribution values.

In [ ]:
# A custom function to calculate probability distribution function 
def pdf(x,m,std):      
    y_out = 1/(std * np.sqrt(2 * np.pi)) * np.exp( - (x - m)**2 / (2 * std**2)) 
    return y_out 

# To generate an array of x-values 
x = np.arange(-5, 12, 0.1) 

# To generate an array of 
# y-values using corresponding x-values 
y = pdf(x,4,3) 

# Plotting the bell-shaped curve 
plt.plot(x, y) 
plt.show()

Enter the correct values in the equation below to calculate the percentage of noise datapoints falling
+/- 1 SDs from the mean. **Remove answers**

In [ ]:
from scipy.stats import norm

round(100 * (norm.cdf(7, loc=4, scale=3) - norm.cdf(1, loc=4, scale=3)),2)

Enter the correct values in the equation below to calculate the percentage of noise datapoints falling
+/- 2 SDs from the mean.

In [ ]:
from scipy.stats import norm

round(100 * (norm.cdf(10, loc=4, scale=3) - norm.cdf(-2, loc=4, scale=3)),2)

Enter the correct values in the equation below to calculate the percentage of noise datapoints falling
+/- 3 SDs from the mean.

In [ ]:
from scipy.stats import norm

round(100 * (norm.cdf(13, loc=4, scale=3) - norm.cdf(-5, loc=4, scale=3)),2)

### <font color=red>DSP.04.G2.c) Calculating percentages</font>

In the two examples above, the mean and variance values differed, but the percentage of noise datapoints
falling within the 1, 2, or 3 SD boundaries was identical. Write a sentence or two about why this
will always be the case if the data are normally distributed.

### <font color=red>DSP.04.G2.d) Calculating percentages</font>

The values are already calculated above, but fill in the blanks below and spell it out.

These are handy facts to have on hand. In a normal distribution...

____ percentage of cases fall +/- 1 SD from the mean.

____ percentage of cases fall +/- 2 SD from the mean.

____ percentage of cases fall +/- 3 SD from the mean.